In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ee
import geemap

PROJECT_ID = "ee-a01571000"

ee.Authenticate()
ee.Initialize(project=PROJECT_ID)

print(ee.String("Succesful connection to Google Earth Engine!").getInfo())  # type: ignore

Succesful connection to Google Earth Engine!


In [3]:
map = geemap.Map()
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [11]:
# Dataset de imágenes obtenido de
# https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1#description

img_set = ee.ImageCollection("LANDSAT/LC08/C02/T1")  # type: ignore
bands = img_set.select(["B4", "B3", "B5"])

# B3 - Green
# B4 - Red
# B5 - NIR (Near Infrared)

# Parece que normalmente la variable L (factor de ajuste del suelo) se pone como L = 0.5
# según en la página 4 https://catalogos.conae.gov.ar/landsat8/Docs/IndicesEspectralesDerivadosDeLandsat8.pdf

visualization = {
    min: 0.0,
    max: 30000.0,
}

# Actualizar el mapa interactivo
map.setCenter(-100.4, 25.73, 10)
map.addLayer(bands, visualization, "Green + Red + NIR")

In [12]:
# Define the Area of Interest (AOI)
# lat, long de esquina abajo izquiera y lat, long de esquina arriba derecha
# Se crea un triangulo con estos dos puntos, siendo la área

aoi = ee.Geometry.Rectangle([-100.3543, 25.5664, -100.3049, 25.5989]) ### Aquí van las coordenadas para el área



In [14]:
# Filter the Image Collection by AOI
filtered_images = img_set.filterBounds(aoi)



In [15]:
# Count the number of images in the filtered collection
image_count = filtered_images.size().getInfo()
print(f"Number of images available: {image_count}")


Number of images available: 249


In [16]:

# Get the oldest and newest images based on the 'system:time_start' property
oldest_image = filtered_images.sort('system:time_start').first()
newest_image = filtered_images.sort('system:time_start', False).first()

# Get the dates of the oldest and newest images
oldest_date = ee.Date(oldest_image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
newest_date = ee.Date(newest_image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

print(f"Oldest image date: {oldest_date}")
print(f"Newest image date: {newest_date}")

Oldest image date: 2013-04-04
Newest image date: 2024-08-04
